# Use Linear Regression model to predict stock daily return rates

## Prepare data

In [1]:
import numpy as np
import pandas as pd
import warnings
import sys
import os

warnings.simplefilter(action='ignore', category=FutureWarning) # Ignore all future warning

In [2]:
df = pd.read_csv('../data/processed/VNINDEX_add_features.csv')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6423 entries, 0 to 6422
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   DTYYYYMMDD            6423 non-null   object 
 1   Ticker                6423 non-null   object 
 2   Open                  6423 non-null   float64
 3   High                  6423 non-null   float64
 4   Low                   6423 non-null   float64
 5   Close                 6423 non-null   float64
 6   Volume                6423 non-null   float64
 7   Outlier               6423 non-null   bool   
 8   daily_returns         6423 non-null   float64
 9   monthly_returns       6423 non-null   float64
 10  yearly_returns        6423 non-null   float64
 11  Net_advances          6423 non-null   int64  
 12  A/D                   6423 non-null   float64
 13  Schultz               6423 non-null   float64
 14  EMA19_net_adv         6423 non-null   float64
 15  EMA39_net_adv        

In [3]:
## Caculate the correlation matrix of the features
corr_df = df.copy()
corr_df = corr_df.drop(columns=['DTYYYYMMDD','Ticker','Open','High','Low','Volume','Close','Outlier','monthly_returns','yearly_returns'])
corr_matrix = corr_df.corr()

## Display the correlation matrix
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)  # ngăn DataFrame xuống dòng
pd.set_option('display.notebook_repr_html', True)
print(corr_matrix)


                      daily_returns  Net_advances       A/D   Schultz  EMA19_net_adv  EMA39_net_adv  McClellan_Oscillator      TRIN  StockAboveMA50       MA5      EMA5      MA10     EMA10      MA20     EMA20      MA50     EMA50     MA100    EMA100     MA200    EMA200
daily_returns              1.000000      0.629682  0.385782  0.360136       0.304914       0.221159              0.357898 -0.045105        0.078571 -0.025212 -0.021846 -0.030325 -0.027663 -0.034325 -0.032604 -0.040646 -0.038338 -0.042650 -0.041812 -0.044733 -0.044045
Net_advances               0.629682      1.000000  0.499066  0.634140       0.443493       0.321071              0.521660 -0.027070        0.148104  0.025248  0.027990  0.022683  0.024633  0.021810  0.022581  0.019807  0.020557  0.018636  0.019296  0.017505  0.019351
A/D                        0.385782      0.499066  1.000000  0.428697       0.156179       0.102137              0.203736 -0.022393        0.111282  0.114891  0.116383  0.115240  0.116160  0.11777

In [4]:
# So when we analyze the correlation matrix, we decide to use the features: Net_advances,A/D,Schultz, and McClellan_Oscillator
selected_features = ['Net_advances','A/D','Schultz','McClellan_Oscillator']
target_feature = ['daily_returns']
## Create new numpy array with the selected features
X_selected = df[selected_features].to_numpy()
y_selected = df[target_feature].to_numpy()


In [5]:
print(X_selected.shape)
print(y_selected.shape)


(6423, 4)
(6423, 1)


## Train and predict with LinearRegression model

In [6]:
# Get the absolute path of the `src` folder
src_path = os.path.abspath(os.path.join(os.getcwd(), "..", "src"))
# Add `src` to the system path
sys.path.insert(0, src_path)
# Import
from models.LinearRegression import LinearRegression

model = LinearRegression()

# Train the model
len_X_selected = len(X_selected)
y_pred_set = [[] for _ in range(len_X_selected+3)]

# Predict the next n days
number_of_days_to_predict = 3
for i in range(len_X_selected-number_of_days_to_predict):
    if i < 30:
        continue
    X_train = X_selected[(i-30):i,]
    y_train = y_selected[(i-30):i,]
    model.fit(X_train,y_train)
    y_pred = model.predict(X_selected[(i+1):(i+1+number_of_days_to_predict),])
    for j in range(number_of_days_to_predict):
        if i+j+1 < len_X_selected:
            y_pred_set[i+j+1].append(y_pred[j])

# Calculate the mean of the predicted values
y_pred_set = y_pred_set[:len_X_selected]
y_predict = []
for i in range(len_X_selected):
    if i < 30:
        y_predict.append(np.nan)
        continue
    y_predict.append(np.mean(y_pred_set[i]))

# Convert the predicted values to a numpy array
y_predict = np.array(y_predict).reshape(-1,1)

# Calculate the R-squared score
score = model.score(y_selected,y_predict)
print(score)


0.5461581262367672


c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [7]:
np.array(df['Close'])
print(np.array(df['Close'][1]))

101.55


## Trade with the predict

In [8]:
from trade_stocks.trade import Trading

trading = Trading()
trade_signal = trading.generate_trade_signal(y_predict)
print(trade_signal)
finally_capital = trading.execute_trade(trade_signal,np.array(df['Close']))
print(finally_capital)
performance = trading.performance()


[ 0.  0.  0. ... -1. -1.  1.]
1990867.5859561935
Total profit: -9132.414043806493
Total trade number: 2429
Win rate: 0.492383696994648
Profit factor: 0.9050236258528692
